In [1]:
import numpy as np
from sympy import symbols, And, Not, simplify, solve, reduce_inequalities
%run constants.ipynb
%run li_handler.ipynb

In [2]:
def no_loser(a,b,c):
    return [simplify(i) for i in [a>0, b>0, c>0]]

In [3]:
def parentStates(a,b,c, r=V):
    return [s for s in 
            [(2*a,b-a,c), (2*a,b,c-a),
            (a-b,2*b,c), (a,2*b,c-b),
            (a-c,b,2*c), (a,b-c,2*c)] 
            if feasible(*no_loser(*s), r=r)]

In [125]:
@memoized
def coef(n):
    return Fraction(1, 6**n)

In [5]:
def concat(ls):
    ''' Flatten 2D list. '''
    return [j for l in ls for j in l]

In [6]:
import csv
from itertools import islice

def read(filename, min_n=1, max_n=5):
    hs = {}
    with open(filename, 'r') as fp:
        reader = islice(csv.reader(fp), min_n-1, max_n)
        for row in reader:
            n, c, rs = row
            # parse entries
            hs[int(n)] = (int(c), sympify(rs))
    return hs

In [7]:
def h_mult(n,a,b,c):
    return non_redundant(*no_loser(a,b,c), r=V)

def mult_inds(inds, prereqs, r=V):
    '''
    prereqs * sum(inds), keeping only non-redundant indicators in `prereqs` given restriction `r`.
    '''
    return [i + [p for p in non_redundant(*prereqs, r=V+i)] for i in inds]

@memoized
def _h(n,a,b,c):
    ''' Returns the additive indicators of h_n(a,b,c), as a 2D list. '''
    # constant coef = (1/6)^n
    # so we only keep track of the indicators to be summed up
    if n == 1:
        return mult_inds([[simplify(a<=b)], [simplify(a<=c)]], h_mult(1,a,b,c))
    # list of inequalities representing the region Rn
    return mult_inds(concat([_h(n-1, *s) for s in parentStates(a,b,c)]), 
                     h_mult(n,a,b,c))

@memoized
def h(n,a,b,c, regenerate):
    if regenerate:
        return reduce(_h(n,a,b,c), r=V)
    else:
        if (a,b,c) == (y,x,z):
            return read(H_YXZ_CACHE, max_n=n)[n]
        else:
            return read(H_XYZ_CACHE, max_n=n)[n]

# Use xyz

In [11]:
import gurobipy as gp
from gurobipy import GRB
from itertools import repeat

In [1702]:
n = 3
H_XYZ = read(H_XYZ_CACHE, min_n=1, max_n=n)
H_YXZ = read(H_YXZ_CACHE, min_n=1, max_n=n)
num_pos_per_n = [len(H_XYZ[i][1]) for i in range(1,n+1)]
num_neg_per_n = [len(H_YXZ[i][1]) for i in range(1,n+1)]
# num_pos_per_n = [len(concat(H_XYZ[i][1])) for i in range(1,n+1)]
# num_neg_per_n = [len(concat(H_YXZ[i][1])) for i in range(1,n+1)]
num_pos_per_n

[0, 5, 31]

In [1703]:
num_pos = sum(num_pos_per_n)  # number of positive compound indicators
num_neg = sum(num_neg_per_n)
num_pos

36

In [1704]:
pos_coefs = concat([repeat(j, k) for j,k in zip([coef(i) for i in range(1,n+1)], num_pos_per_n)])
neg_coefs = concat([repeat(j, k) for j,k in zip([-coef(i) for i in range(1,n+1)], num_neg_per_n)])
pos_coefs

[Fraction(1, 36),
 Fraction(1, 36),
 Fraction(1, 36),
 Fraction(1, 36),
 Fraction(1, 36),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216),
 Fraction(1, 216)]

In [1705]:
const = sum([coef(i)*(H_XYZ[i][0] - H_YXZ[i][0]) for i in range(1,n+1)])
const

Fraction(1, 6)

In [1706]:
all_pos_comp_inds = concat([H_XYZ[i][1] for i in range(1,n+1)])
all_neg_comp_inds = concat([H_YXZ[i][1] for i in range(1,n+1)])
all_pos_comp_inds

[[3*x <= y],
 [2*x <= z],
 [2*x <= y],
 [3*x <= z],
 [x <= -y + z],
 [7*x <= y],
 [4*x <= z, 3*x < y],
 [5*x <= y],
 [6*x <= z],
 [5*x <= 3*y, y < 3*x],
 [3*x <= y + z, y < 3*x],
 [2*x <= y],
 [x <= -y + z],
 [3*x <= y + z, z < 2*x],
 [z < 2*x],
 [6*x <= y],
 [5*x <= z, 2*x < y],
 [4*x <= y],
 [7*x <= z],
 [y < 2*x],
 [3*x <= y + z, y < 2*x],
 [x < -y + z],
 [3*x <= -y + z],
 [3*x <= y + z, z < 3*x],
 [5*x <= 3*z, z < 3*x],
 [2*x <= z, -y + z < x],
 [3*x <= 2*y],
 [2*x <= -y + z],
 [x < -y + z],
 [3*x <= -y + z],
 [3*y < z],
 [x <= -3*y + z],
 [-y + z < x],
 [x <= -3*y + 3*z, -y + z < x],
 [x <= 3*y - z],
 [x <= -2*y + 2*z, z < 3*y]]

In [1707]:
model = gp.Model(env=gp.Env())
# model.Params.LazyConstraints = 1
model.Params.OutputFlag = 1
model.Params.LogToConsole = 1
# model.Params.MIPFocus = 1
# model.Params.Heuristics = 1
# model.Params.CliqueCuts = 1
# model.Params.FeasibilityTol = 1e-4
# model.Params.IntFeasTol = 1e-9
# model.Params.MarkowitzTol = 1e-4
# model.Params.NodeLimit = 10000

gx = model.addVar(lb=1, vtype='I', name='x')
gy = model.addVar(lb=2, vtype='I', name='y')
gz = model.addVar(lb=3, vtype='I', name='z')

# gx = model.addVar(lb=0, name='x')
# gy = model.addVar(lb=1, name='y')
# gz = model.addVar(lb=2, name='z')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02


In [1708]:
from sympy import symbols, Add
from gurobipy import Model, LinExpr

# Define SymPy variables
x,y,z = symbols('x y z')

# Mapping between SymPy and Gurobi variables
SP_TO_GP = {x: gx, y: gy, z: gz}

def to_gp_linexpr(sympy_expr, var_map=SP_TO_GP):
    gurobi_expr = LinExpr()

    if isinstance(sympy_expr, Add):
        for term in sympy_expr.args:
            gurobi_expr.add(sympy_to_gurobi_linexpr(term, var_map))
    elif sympy_expr.is_Mul:
        coeff = 1
        var = None
        for factor in sympy_expr.args:
            if factor.is_number:
                coeff *= float(factor)
            elif factor in var_map:
                var = var_map[factor]
        if var is not None:
            gurobi_expr.addTerms(coeff, var)
        else:
            gurobi_expr.addConstant(coeff)
    elif sympy_expr in var_map:
        gurobi_expr.addTerms(1.0, var_map[sympy_expr])
    elif sympy_expr.is_number:
        gurobi_expr.addConstant(float(sympy_expr))

    return gurobi_expr

In [1709]:
def unify_ineq(ineq):
    ineq = simplify(ineq)
    lhs, rhs, op = ineq.lhs, ineq.rhs, ineq.rel_op
    # Check the inequality type and unify the format
    if op == '>=':
        # lhs >= rhs -> rhs <= lhs -> rhs - lhs <= 0
        return Rel(rhs - lhs, 0, '<=')
    elif op == '>':
        # lhs > rhs -> lhs - rhs > 0
        return Rel(lhs - rhs, 0, '>')
    elif op == '<=':
        # lhs <= rhs -> lhs - rhs <= 0
        return Rel(lhs - rhs, 0, '<=')
    elif op == '<':
        # lhs < rhs -> rhs > lhs -> rhs - lhs > 0
        return Rel(rhs - lhs, 0, '>')
    else:
        raise ValueError(f"Unhandled inequality type: {op}")

In [1710]:
M = 300
# to non-stricten strict inequalities
# expressions are in terms of x,y,z, which are all ints
EPS = 1
def add_ind_constr(ind, var):
    r = unify_ineq(ind)  # to the form <= 0 or > 0
    lhs = to_gp_linexpr(r.lhs)
    if r.rel_op == '>':
        model.addConstr(lhs >= EPS - M * (1 - var))
        model.addConstr(lhs <= M * var - EPS)
    elif r.rel_op == '<=':
        model.addConstr(lhs <= M * (1 - var))
        model.addConstr(lhs >= EPS - M * var)
    else:
        raise ValueError 

In [1711]:
# for each compound indicator group, restrict their sum to be [0,1]
def add_comp_ind(comp_ind):
    ''' Adds variables and constraints for a compound indicator. '''
    I = model.addVar(vtype=GRB.BINARY, name='main_ind')
    component_vars = model.addVars(len(comp_ind), vtype=GRB.BINARY, name='component_ind').values()
    
    for c in component_vars:
        model.addConstr(I <= c)
    model.addConstr(I >= gp.quicksum(component_vars) - (len(component_vars)-1))
    
    # big-M method to enforce semantics of inequalities
    # if the inequality holds, then the corresponding var is true
    for ineq, c in zip(comp_ind, component_vars):
        add_ind_constr(ineq, c)
    
    return I, list(component_vars)

In [1712]:
def add_single_ind(ind):
    I = model.addVar(vtype=GRB.BINARY, name='single_ind')
    add_ind_constr(ind[0], I)
    return (I,)  # no component variables

In [1713]:
def add_ind(ind):
    return add_single_ind(ind) if len(ind) == 1 else add_comp_ind(ind)

In [1714]:
pos_inds = [add_ind(i) for i in all_pos_comp_inds]
neg_inds = [add_ind(i) for i in all_neg_comp_inds]
pos_inds

[(<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),


In [1715]:
obj = gp.quicksum([pos_coefs[i]*pos_inds[i][0] for i in range(num_pos)]) + \
        gp.quicksum([neg_coefs[i]*neg_inds[i][0] for i in range(num_neg)]) + const - thresh(n)

In [1716]:
model.setObjective(obj, GRB.MINIMIZE)
f_constr = model.addConstr(obj <= 0, name='non_pos_f')
# constraints for V
model.addConstr(gx <= gy - 1, name='V1')
model.addConstr(gy <= gz - 1, name='V2')

<gurobi.Constr *Awaiting Model Update*>

In [1717]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 240 rows, 111 columns and 807 nonzeros
Model fingerprint: 0xf971a382
Variable types: 0 continuous, 111 integer (108 binary)
Coefficient statistics:
  Matrix range     [5e-03, 3e+02]
  Objective range  [5e-03, 3e-02]
  Bounds range     [1e+00, 3e+00]
  RHS range        [7e-02, 3e+02]
Presolve removed 32 rows and 0 columns
Presolve time: 0.00s
Presolved: 208 rows, 111 columns, 733 nonzeros
Variable types: 0 continuous, 111 integer (108 binary)
Found heuristic solution: objective -0.0120370

Root relaxation: objective -6.600999e-02, 28 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -0.06601    0   16   -0.01204   -0.06601 

In [1718]:
pos_ind_main_vars = [i[0] for i in pos_inds]
neg_ind_main_vars = [i[0] for i in neg_inds]

In [1719]:
def analyse_result():
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found:")
        for v in model.getVars():
            print(f"{v.varName}: {v.x}, {not (not v.x)}")
        print(f"Objective value: {model.objVal}")
    elif model.status == GRB.UNBOUNDED:
        print('The model cannot be solved because it is unbounded')
    elif model.status != GRB.INF_OR_UNBD and model.status != GRB.INFEASIBLE:
        print('Optimization was stopped with status %d' % model.status)
    else:
        # Relax the bounds and try to make the model feasible
        print('The model is infeasible; relaxing the bounds')
        # can only relax thresh constraint
#         relaxable_vars = pos_ind_main_vars + neg_ind_main_vars
#         pen = [1] * len(relaxable_vars)
#         model.feasRelax(1, False, 
#                         relaxable_vars, pen, pen, 
#                         [thresh_constr], 
#                         None)
        model.feasRelax(1, False, None, None, None, [f_constr], [0.1])
        model.optimize()

In [1720]:
analyse_result()

Optimal solution found:
x: 4.0, True
y: 5.0, True
z: 6.0, True
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True
single_ind: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True
single_ind: 1.0, True
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 1.0, True
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True
single_ind: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True


In [1425]:
analyse_result()

Optimal solution found:
x: 8.0, True
y: 9.0, True
z: 10.0, True
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True
single_ind: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True
single_ind: 1.0, True
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 1.0, True
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True
single_ind: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True

main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 0.0, False
single_ind: 0.0, False
single_ind: 1.0, True
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True
single_ind: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True
component_ind[2]: 0.0, False
main_ind: 0.0, False
component_ind[0]: 0.0, False
component_ind[1]: 1.0, True
main_ind: 0.0, False
component_ind[0]: 1.0, True
component_i

In [1721]:
# 2: 3,4,5
# 3: 4,5,6
# 4: 0.5, 1.5, 4.5
def eval_f(n,a,b,c):
    tot = 0.
    vals = {x:a, y:b, z:c}
    for i in range(1,n+1):
        pos_c, pos_rs = H_XYZ[i]
        neg_c, neg_rs = H_YXZ[i]
#         print(sum([sub(r, vals) for r in neg_rs]))
        tot += coef(i) * (pos_c - neg_c + sum([sub(r, vals) for r in pos_rs]) - sum([sub(r, vals) for r in neg_rs]))
    return tot - thresh(n)

In [1723]:
eval_f(3,4,5,6)

-0.016666666666666677

# Verify

In [1248]:
model.computeIIS()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads


Computing Irreducible Inconsistent Subsystem (IIS)...

           Constraints          |            Bounds           |  Runtime
      Min       Max     Guess   |   Min       Max     Guess   |
--------------------------------------------------------------------------
        0     14827         -         0         3         -           0s
        0      4192        10         0         3         -           5s
        8      1114        19         0         3         -          10s
       27        27        27         1         1         1          13s

IIS computed: 27 constraints, 1 bounds
IIS runtime: 13.06 seconds (8.62 work units)


In [1249]:
model.write("model5.ilp")  # This writes the IIS to a file

# Print constraints that are part of the IIS
for constr in model.getConstrs():
    if constr.IISConstr:
        print(f"Infeasible constraint: {constr.constrName}")

Infeasible constraint: R7
Infeasible constraint: R9
Infeasible constraint: R22
Infeasible constraint: R64
Infeasible constraint: R98
Infeasible constraint: R102
Infeasible constraint: R104
Infeasible constraint: R3723
Infeasible constraint: R7726
Infeasible constraint: R7732
Infeasible constraint: R7736
Infeasible constraint: R7738
Infeasible constraint: R7749
Infeasible constraint: R7752
Infeasible constraint: R7756
Infeasible constraint: R7770
Infeasible constraint: R7774
Infeasible constraint: R7776
Infeasible constraint: R7785
Infeasible constraint: R7787
Infeasible constraint: R7798
Infeasible constraint: R7807
Infeasible constraint: R7818
Infeasible constraint: R10804
Infeasible constraint: R10805
Infeasible constraint: non_pos_f
Infeasible constraint: V1


In [124]:
for _x in range(1, 41):
    for _y in range(_x+1, 41):
        for _z in range(_y+1, 41):
            if eval_f(n,_x,_y,_z) <= 0:
                print("Failed!")
            else:
                print("no")

no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
n

no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
n

no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
n

no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
n

# General constraints

In [57]:
import gurobipy as gp
from gurobipy import GRB
from itertools import repeat

In [58]:
n = 4
H_XYZ = read(H_XYZ_CACHE, min_n=1, max_n=n)
H_YXZ = read(H_YXZ_CACHE, min_n=1, max_n=n)
num_pos_per_n = [len(H_XYZ[i][1]) for i in range(1,n+1)]
num_neg_per_n = [len(H_YXZ[i][1]) for i in range(1,n+1)]
num_pos_per_n

[0, 5, 31, 163]

In [59]:
num_pos = sum(num_pos_per_n)  # number of positive compound indicators
num_neg = sum(num_neg_per_n)
num_pos

199

In [60]:
pos_coefs = concat([repeat(j, k) for j,k in zip([coef(i) for i in range(1,n+1)], num_pos_per_n)])
neg_coefs = concat([repeat(j, k) for j,k in zip([-coef(i) for i in range(1,n+1)], num_neg_per_n)])

In [61]:
const = sum([coef(i)*(H_XYZ[i][0] - H_YXZ[i][0]) for i in range(1,n+1)])
const

0.16666666666666666

In [62]:
all_pos_comp_inds = concat([H_XYZ[i][1] for i in range(1,n+1)])
all_neg_comp_inds = concat([H_YXZ[i][1] for i in range(1,n+1)])
all_pos_comp_inds

[[3*x <= y],
 [2*x <= z],
 [2*x <= y],
 [3*x <= z],
 [x <= -y + z],
 [7*x <= y],
 [4*x <= z, 3*x < y],
 [5*x <= y],
 [6*x <= z],
 [5*x <= 3*y, y < 3*x],
 [3*x <= y + z, y < 3*x],
 [2*x <= y],
 [x <= -y + z],
 [3*x <= y + z, z < 2*x],
 [z < 2*x],
 [6*x <= y],
 [5*x <= z, 2*x < y],
 [4*x <= y],
 [7*x <= z],
 [y < 2*x],
 [3*x <= y + z, y < 2*x],
 [x < -y + z],
 [3*x <= -y + z],
 [3*x <= y + z, z < 3*x],
 [5*x <= 3*z, z < 3*x],
 [2*x <= z, -y + z < x],
 [3*x <= 2*y],
 [2*x <= -y + z],
 [x < -y + z],
 [3*x <= -y + z],
 [3*y < z],
 [x <= -3*y + z],
 [-y + z < x],
 [x <= -3*y + 3*z, -y + z < x],
 [x <= 3*y - z],
 [x <= -2*y + 2*z, z < 3*y],
 [15*x <= y],
 [8*x <= z, 7*x < y],
 [11*x <= y],
 [12*x <= z, 3*x < y],
 [13*x <= 3*y, y < 7*x],
 [7*x <= y + z, y < 7*x, 3*x < y],
 [5*x <= y],
 [x <= -y + z, 3*x < y],
 [7*x <= y + z, z < 4*x, 3*x < y],
 [4*x <= 3*z, z < 4*x, 3*x < y],
 [13*x <= y],
 [10*x <= z, 5*x < y],
 [9*x <= y],
 [14*x <= z],
 [7*x <= 3*y, y < 5*x],
 [7*x <= y + z, y < 5*x],
 [3*x

In [63]:
model = gp.Model(env=gp.Env())
model.Params.OutputFlag = 1
model.Params.LogToConsole = 1
model.Params.MIPFocus = 1
# model.Params.Heuristics = 1
model.Params.CliqueCuts = 1
# model.Params.FeasibilityTol = 1e-4
model.Params.IntFeasTol = 1e-9
# model.Params.MarkowitzTol = 1e-4
# model.Params.NodeLimit = 10000

# gx = model.addVar(lb=1, vtype='I', name='x')
# gy = model.addVar(lb=2, vtype='I', name='y')
# gz = model.addVar(lb=3, vtype='I', name='z')

gx = model.addVar(lb=1, name='x')
gy = model.addVar(lb=2, name='y')
gz = model.addVar(lb=3, name='z')

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02
Set parameter MIPFocus to value 1
Set parameter CliqueCuts to value 1
Set parameter IntFeasTol to value 1e-09


In [79]:
from sympy import symbols, Add
from gurobipy import Model, LinExpr, and_

# Define SymPy variables
x,y,z = symbols('x y z')

# Mapping between SymPy and Gurobi variables
SP_TO_GP = {x: gx, y: gy, z: gz}

def to_gp_linexpr(sympy_expr, var_map=SP_TO_GP):
    gurobi_expr = LinExpr()

    if isinstance(sympy_expr, Add):
        for term in sympy_expr.args:
            gurobi_expr.add(to_gp_linexpr(term, var_map))
    elif sympy_expr.is_Mul:
        coeff = 1
        var = None
        for factor in sympy_expr.args:
            if factor.is_number:
                coeff *= float(factor)
            elif factor in var_map:
                var = var_map[factor]
        if var is not None:
            gurobi_expr.addTerms(coeff, var)
        else:
            gurobi_expr.addConstant(coeff)
    elif sympy_expr in var_map:
        gurobi_expr.addTerms(1.0, var_map[sympy_expr])
    elif sympy_expr.is_number:
        gurobi_expr.addConstant(float(sympy_expr))

    return gurobi_expr

In [80]:
EPS = 1
def unify_ineq(ineq):
    ineq = simplify(ineq)
    lhs, rhs, op = ineq.lhs, ineq.rhs, ineq.rel_op
    # Check the inequality type and unify the format
    if op == '>=':
        # lhs >= rhs -> lhs - rhs >= 0
        return Rel(lhs - rhs, 0, '>=')
    elif op == '>':
        # lhs > rhs -> lhs - rhs > 0 => lhs - rhs >= EPS
        return Rel(lhs - rhs - EPS, 0, '>=')
    elif op == '<=':
        # lhs <= rhs -> lhs - rhs <= 0 -> rhs - lhs >= 0
        return Rel(rhs - lhs, 0, '>=')
    elif op == '<':
        # lhs < rhs -> rhs > lhs -> rhs - lhs > 0 -> rhs - lhs >= EPS
        return Rel(rhs - lhs - EPS, 0, '>=')
    else:
        raise ValueError(f"Unhandled inequality type: {op}")

In [81]:
def add_ind_constr(ineq, name='ind'):
    ''' Add indicator constraint for a single inequality'''
    r = unify_ineq(ineq)  # to the form >= 0
    lhs = to_gp_linexpr(r.lhs)
    v = model.addVar(vtype=GRB.BINARY, name=name)
    model.addConstr((v == 1) >> (lhs >= 0))
    model.addConstr((v == 0) >> (lhs <= -EPS))  # lhs < 0
    return v

In [82]:
# for each compound indicator group, restrict their sum to be [0,1]
def add_ind_constrs(*ineqs):
    ''' Adds variables and constraints for a compound indicator. '''
    if len(ineqs) == 1:
        return (add_ind_constr(ineqs[0], name='single_ind'),)
    
    I = model.addVar(vtype=GRB.BINARY, name='main_ind')
    component_vars = [add_ind_constr(i, name='component_ind') for i in ineqs]
    model.addConstr(I == and_(component_vars), name='andconstr')
    return I, component_vars

In [68]:
pos_inds = [add_ind_constrs(*i) for i in all_pos_comp_inds]
neg_inds = [add_ind_constrs(*i) for i in all_neg_comp_inds]
pos_inds

[(<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,
  [<gurobi.Var *Awaiting Model Update*>,
   <gurobi.Var *Awaiting Model Update*>]),
 (<gurobi.Var *Awaiting Model Update*>,),
 (<gurobi.Var *Awaiting Model Update*>,),


In [69]:
obj = gp.quicksum([pos_coefs[i]*pos_inds[i][0] for i in range(num_pos)]) + \
        gp.quicksum([neg_coefs[i]*neg_inds[i][0] for i in range(num_neg)]) + const - thresh(n)

In [73]:
model.setObjective(obj, GRB.MINIMIZE)
# f_constr = model.addConstr(obj <= 0, name='non_pos_f')
# constraints for V
model.addConstr(gx <= gy - 1, name='V1')
model.addConstr(gy <= gz - 1, name='V2')

<gurobi.Constr *Awaiting Model Update*>

In [74]:
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 907 columns and 8 nonzeros
Model fingerprint: 0x96edb5d8
Model has 1565 general constraints
Variable types: 3 continuous, 904 integer (904 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-04, 3e-02]
  Bounds range     [1e+00, 3e+00]
  RHS range        [1e+00, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 2e+01]

MIP start from previous solve produced solution with objective 0.0240741 (0.02s)
Loaded MIP start from previous solve with objective 0.0240741

Presolve added 2363 rows and 1983 columns
Presolve time: 0.03s
Presolved: 2367 rows, 2890 columns, 6874 nonzeros
Presolved model has 1322 SOS constraint(s)
Variable types: 1325 continuous, 1565 integer (1565 binary)

Root relaxation: objective -2.266975e-01, 136

In [1902]:
pos_ind_main_vars = [i[0] for i in pos_inds]
neg_ind_main_vars = [i[0] for i in neg_inds]

In [44]:
def analyse_result(m=model):
    if model.status == GRB.OPTIMAL:
        print("Optimal solution found:")
        for v in model.getVars():
            print(f"{v.varName}: {v.x}, {not (not v.x)}")
        print(f"Objective value: {model.objVal}")
    elif model.status == GRB.UNBOUNDED:
        print('The model cannot be solved because it is unbounded')
    elif model.status != GRB.INF_OR_UNBD and model.status != GRB.INFEASIBLE:
        print('Optimization was stopped with status %d' % model.status)
    else:
        # Relax the bounds and try to make the model feasible
        print('The model is infeasible; relaxing the bounds')
        # can only relax thresh constraint
#         relaxable_vars = pos_ind_main_vars + neg_ind_main_vars
#         pen = [1] * len(relaxable_vars)
#         model.feasRelax(1, False, 
#                         relaxable_vars, pen, pen, 
#                         [thresh_constr], 
#                         None)
        model.feasRelax(1, False, None, None, None, [f_constr], [0.1])
        model.optimize()

In [72]:
analyse_result()

Optimal solution found:
x: 9.0, True
y: 10.0, True
z: 14.0, True
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind: 0.0, False
component_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind: 0.0, False
component_ind: 1.0, True
main_ind: 0.0, False
component_ind: 0.0, False
component_ind: 1.0, True
single_ind: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind: 0.0, False
component_ind: 1.0, True
single_ind: 1.0, True
single_ind: 0.0, False
main_ind: 0.0, False
component_ind: 0.0, False
component_ind: 0.0, False
single_ind: 0.0, False
single_ind: 0.0, False
single_ind: 1.0, True
main_ind: 0.0, False
component_ind: 0.0, False
component_ind: 1.0, True
single_ind: 0.0, False
single_ind: 0.0, False
main_ind: 0.0, False
component_ind: 0.0, False
component_ind: 1.0, True
main_ind: 0.0, False
component_ind: 0.0,

In [16]:
def eval_f(n,a,b,c):
    tot = 0.
    vals = {x:a, y:b, z:c}
    for i in range(1,n+1):
        pos_c, pos_rs = H_XYZ[i]
        neg_c, neg_rs = H_YXZ[i]
#         print(sum([sub(r, vals) for r in neg_rs]))
        tot += coef(i) * (pos_c - neg_c + sum([sub(r, vals) for r in pos_rs]) - sum([sub(r, vals) for r in neg_rs]))
    return tot - thresh(n)

In [75]:
eval_f(4,9,10,14)

0.024074074074074067

In [1842]:
model.write('modelB.lp')

In [111]:
n = 4
H_XYZ = read(H_XYZ_CACHE, min_n=1, max_n=n)
H_YXZ = read(H_YXZ_CACHE, min_n=1, max_n=n)
model.reset(1)
model = gp.Model(env=gp.Env())

gx = model.addVar(lb=1, vtype='I', name='x')
gy = model.addVar(lb=2, vtype='I', name='y')
gz = model.addVar(lb=3, vtype='I', name='z')

Discarded solution information including additional information
Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-02


In [112]:
# Mapping between SymPy and Gurobi variables
SP_TO_GP = {x: gx, y: gy, z: gz}

def to_gp_linexpr(sympy_expr, var_map=SP_TO_GP):
    gurobi_expr = LinExpr()

    if isinstance(sympy_expr, Add):
        for term in sympy_expr.args:
            gurobi_expr.add(to_gp_linexpr(term, var_map))
    elif sympy_expr.is_Mul:
        coeff = 1
        var = None
        for factor in sympy_expr.args:
            if factor.is_number:
                coeff *= float(factor)
            elif factor in var_map:
                var = var_map[factor]
        if var is not None:
            gurobi_expr.addTerms(coeff, var)
        else:
            gurobi_expr.addConstant(coeff)
    elif sympy_expr in var_map:
        gurobi_expr.addTerms(1.0, var_map[sympy_expr])
    elif sympy_expr.is_number:
        gurobi_expr.addConstant(float(sympy_expr))

    return gurobi_expr

In [113]:
coeff = coef(n)
pos_inds = [add_ind_constrs(*i) for i in H_XYZ[n][1]]
neg_inds = [add_ind_constrs(*i) for i in H_YXZ[n][1]]
const = coeff * (H_XYZ[n][0] - H_YXZ[n][0])

In [114]:
model.addConstr(gx <= gy - 1, name='V1')
model.addConstr(gy <= gz - 1, name='V2')

<gurobi.Constr *Awaiting Model Update*>

In [115]:
obj = coeff * (gp.quicksum([i[0] for i in pos_inds]) - gp.quicksum([i[0] for i in neg_inds]) + const)
model.setObjective(obj, GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 799 columns and 4 nonzeros
Model fingerprint: 0x0e0189e7
Model has 1370 general constraints
Variable types: 0 continuous, 799 integer (796 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [8e-04, 8e-04]
  Bounds range     [1e+00, 3e+00]
  RHS range        [1e+00, 1e+00]
  GenCon rhs range [1e+00, 1e+00]
  GenCon coe range [1e+00, 2e+01]
Presolve added 2073 rows and 1722 columns
Presolve time: 0.03s
Presolved: 2075 rows, 2521 columns, 6037 nonzeros
Presolved model has 1148 SOS constraint(s)
Variable types: 0 continuous, 2521 integer (1370 binary)
Found heuristic solution: objective -0.0007716
Found heuristic solution: objective -0.0015432

Root relaxation: objective -1.118827e-01, 1204 iterations, 0.01 seconds (0.00 work units)

    Node

In [116]:
analyse_result()

Optimal solution found:
x: 7.0, True
y: 8.0, True
z: 23.0, True
single_ind: -0.0, False
main_ind: -0.0, False
component_ind: -0.0, False
component_ind: 0.0, False
single_ind: -0.0, False
main_ind: -0.0, False
component_ind: 0.0, False
component_ind: -0.0, False
main_ind: -0.0, False
component_ind: -0.0, False
component_ind: 1.0, True
main_ind: -0.0, False
component_ind: -0.0, False
component_ind: 1.0, True
component_ind: -0.0, False
single_ind: -0.0, False
main_ind: -0.0, False
component_ind: 1.0, True
component_ind: -0.0, False
main_ind: -0.0, False
component_ind: 0.0, False
component_ind: 1.0, True
component_ind: -0.0, False
main_ind: -0.0, False
component_ind: 1.0, True
component_ind: 1.0, True
component_ind: 0.0, False
single_ind: -0.0, False
main_ind: -0.0, False
component_ind: -0.0, False
component_ind: 0.0, False
single_ind: -0.0, False
single_ind: -0.0, False
main_ind: -0.0, False
component_ind: -0.0, False
component_ind: 1.0, True
main_ind: -0.0, False
component_ind: -0.0, Fal

In [119]:
def eval_dh(n,a,b,c):
    vals = {x:a, y:b, z:c}
    pos_c, pos_rs = H_XYZ[n]
    neg_c, neg_rs = H_YXZ[n]
    return coef(n) * (pos_c - neg_c + sum([sub(r, vals) for r in pos_rs]) - sum([sub(r, vals) for r in neg_rs]))

In [120]:
eval_dh(4,7,8,23)

-0.016975308641975308

In [122]:
eval_f(3,4,5,6) + eval_dh(4,7,8,23) + thresh(4)

0.016358024691358018